In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from pandas.api.types import is_numeric_dtype
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
import datetime
from homebrewedFunctions.functions import *
import re
import multiprocess

def omit_variables(dct_of_dfs, omit):
    dct_of_dfs = dct_of_dfs[~dct_of_dfs.index.get_level_values("Name").isin(omit)].dropna()
    return dct_of_dfs

In [2]:
all_keys = pd.read_csv("StateGovFinances2005to2021.csv", index_col = ["State", "Year", "Format"], low_memory =False).sort_index().keys()
pattern = re.compile(r'(\d+)^NAN(\d+)?$')

all_keys = [k for k in all_keys if not pattern.match(k)]
start_exp = all_keys.index("1EXPENDITURE")
start_debt = all_keys.index("1DEBT OUTSTANDING")
keys_dict = {"Revenues": all_keys[:start_exp],
            "Expenditures": [k for k in all_keys[start_exp:start_debt] if "CAPITAL OUTLAY" not in k] +["1CAPITAL OUTLAY"],
            "Debts": [k for k in all_keys[start_debt:] if "CAPITAL OUTLAY" not in k]}
            
stack_dfs = {"Expenditures": pd.read_csv("StateGovFinances2005to2021.csv", index_col = ["State", "Year", "Format"], usecols = ["State", "Year", "Format", "1GENERAL REVENUE", "1DEBT OUTSTANDING"] + keys_dict["Expenditures"], low_memory =False).sort_index(),
             "Revenues":pd.read_csv("StateGovFinances2005to2021.csv", index_col = ["State", "Year", "Format"], usecols = ["State", "Year", "Format", "1EXPENDITURE", "1DEBT OUTSTANDING"] + keys_dict["Revenues"], low_memory =False).sort_index(),
             "Debts":pd.read_csv("StateGovFinances2005to2021.csv", index_col = ["State", "Year", "Format"], usecols = ["State", "Year", "Format", "1EXPENDITURE", "1GENERAL REVENUE"] + keys_dict["Debts"], low_memory =False).sort_index()}

panel_dfs_dict = {}
for rev_exp in stack_dfs.keys():
    panel_dfs_dict[rev_exp] = {}
    for k in stack_dfs[rev_exp].index.get_level_values(2).unique():
        panel_dfs_dict[rev_exp][k] = stack_dfs[rev_exp][stack_dfs[rev_exp].index.get_level_values(2)==k].reset_index().set_index(["State","Year"]).sort_index()
        del panel_dfs_dict[rev_exp][k]["Format"]
    panel_dfs_dict[rev_exp] = {k.replace("amount", "finances"):panel_dfs_dict[rev_exp][k] for k in ["Local government amount",'State & local government amount', 'State government amount']}

for key, dct in panel_dfs_dict.items():
    for k, df in dct.items():
        for col in df.columns:
            if is_numeric_dtype(df[col]):
                df[col] = df[col].fillna(0)

            else:
                try:
                    # print(col, "forced")

                    df[col] = pd.to_numeric(df[col], errors = "coerce").fillna(0).astype(float)
                except:
                    # print(col, "skipped")
                    pass
        if key == "Revenues":
            df["1TOTAL INCOME"] = df["1INDIVIDUAL INCOME"].add(df["1CORPORATE INCOME"])
            df["1PROPERTY AND SPECIAL ASSESSMENTS"] = df[["1PROPERTY", "1SPECIAL ASSESSMENTS"]].sum(axis = 1)
            df["1DEFICIT"] = df["1EXPENDITURE"].sub(df["1GENERAL REVENUE"])
        if key == "Debts":
            df["1NET INDEBTEDNESS"] = df["1DEBT OUTSTANDING"].sub(df["1OTHER THAN INSURANCE TRUST FUNDS"])
            
        # warning indicates that copying dataframe will defragment it;
        #  not sure if this actually fixes the problem
        df = df.copy()                
cpi_code = {"CPI":"CPIAUCSL"}
start = datetime.datetime(1947,1,1)
end = datetime.datetime.now()
cpi = gather_data(cpi_code, start, end, freq = "A").reset_index().rename(columns = {"DATE": "Year"})
cpi["Year"] = pd.to_datetime(cpi["Year"].astype(str).str[:4], format = "%Y")
cpi["Year"] = cpi["Year"].astype(str)
cpi.set_index("Year", inplace = True)
# set real values to 2023 dollars
cpi["CPI"] = cpi["CPI"].div(cpi["CPI"].iloc[-2]).astype(float)
efnagdp = pd.read_csv("EFNAGDPTaxes.csv", parse_dates = ["Year"]).set_index(["State", "Year"]).sort_index()

def set_plot_dfs(panel_dfs_dict, keys, efnagdp, cpi):
    # keys = ["1" + k for k in keys]
    plot_dfs = {}
    for key in panel_dfs_dict.keys():
        plot_dfs[key] = {}
        plot_dfs[key]["Level"] = panel_dfs_dict[key].apply(lambda x: x.mul(10**3) if "PROPORTION" not in x.name else x)
        plot_dfs[key]["Level"].rename(columns = {k:k.replace("1", "").title() for k in plot_dfs[key]["Level"].keys()}, inplace = True)
    #     plot_dfs[key]["Level"]["Total Income"] = plot_dfs[key]["Level"]["Individual Income"].add(plot_dfs[key]["Level"]["Corporate Income"])
    #     plot_dfs[key]["Level"]["Property and Special Assessments"] = plot_dfs[key]["Level"][["Property", "Special Assessments"]].sum(axis = 1)
        plot_dfs[key]["Level"]["GDP"] = efnagdp["GDP"]
        plot_dfs[key]["Level"]["Deficit"] = plot_dfs[key]["Level"]["Expenditure"].sub(plot_dfs[key]["Level"]["General Revenue"])
        plot_dfs[key]["Real Level"] = plot_dfs[key]["Level"].div(cpi["CPI"], level = "Year", axis = 0)
        plot_dfs[key]["Level"]["Population"] = efnagdp["Population"]
        plot_dfs[key]["Percent of General Revenue"] = plot_dfs[key]["Level"].apply(lambda x: pd.to_numeric(x).div(plot_dfs[key]["Level"]["General Revenue"]).mul(100))    
        plot_dfs[key]["Percent of Expenditure"] = plot_dfs[key]["Level"].apply(lambda x: pd.to_numeric(x).div(plot_dfs[key]["Level"]["Expenditure"]).mul(100))    
        plot_dfs[key]["Percent of GDP"] = plot_dfs[key]["Level"].apply(lambda x: pd.to_numeric(x).div(plot_dfs[key]["Level"]["GDP"]).mul(100))
        plot_dfs[key]["Real Value Per Capita"] = plot_dfs[key]["Real Level"].apply(lambda x: pd.to_numeric(x).div(plot_dfs[key]["Level"]["Population"]))    


    return plot_dfs
plot_dfs = {}

for key in panel_dfs_dict.keys():
    plot_dfs[key] = set_plot_dfs(panel_dfs_dict[key], keys_dict[key], efnagdp, cpi)


import os
for revexp_key in plot_dfs.keys():
    for key in plot_dfs[revexp_key].keys():
        try:
            os.mkdir(f"outputs/{key}")
        except:
            pass

regions_df = pd.read_csv("USCensusRegions.csv")#.set_index("State")
scatter_figs = {}
for revexp_key in plot_dfs.keys():
    for key, p_dfs in plot_dfs[revexp_key].items():
        for p_dfskey, df in p_dfs.items():
            map_figs = {}
            title = f"{key}<br>{revexp_key}: {p_dfskey}"
            html_path = f"outputs/{key}/ScatterPlots{revexp_key}{key}{p_dfskey}.html"
            create_scatter_dropdown(df, regions_df=regions_df,
                                    title = title,
                                    filename = html_path, 
                                    show_fig = False)
            for name in df.keys():
                map_figs[name] = create_map(df.reset_index(), name, title = title,time_name = "Year")
            combined_map_fig = combine_map_figs(map_figs, title)
            html_path = f"outputs/{key}/MapPlots{revexp_key}{key}{p_dfskey}.html"
            combined_map_fig.write_html(html_path)

for revexp_key in plot_dfs.keys():
    for key, p_dfs in plot_dfs[revexp_key].items():
        # figs = {k: line_dropdown(dataframe, regions_df) for k, dataframe in p_dfs.items()}
        # fig = dict_of_line_figs_to_dropdown_fig(figs, show_fig = False, use_sliders = True)
        filename = f"outputs/{key}/LinePlotsStateFinances{revexp_key}{key}.html"
        title = f"{key}<br>{revexp_key}"
        fig = aggregated_line_dropdown(p_dfs, regions_df, title = title)
        fig.write_html(filename, config=dict(displayModeBar=True))

In [3]:
# for k in plot_dfs[key].keys():
#     plot_dfs[key][k]["EFNA"] = efnagdp["EFNA"]
#     plot_dfs[key][k]["Unemployment Rate"] = efnagdp["Unemployment Rate"]
#     plot_dfs[key][k]["Labor Force Participation Rate"] = efnagdp["Labor Force Participation Rate"]
#     plot_dfs[key][k]["Population"] = efnagdp["Population"]

In [4]:
start_year = 2005
areas = {"Revenues":{"Revenue Source by Government" : ["General Revenue From Own Sources", "From Federal Government", "From State Government", "From Local Governments"],
                    "Taxes": ["Intergovernmental Revenue", 'Property', 'Sales And Gross Receipts',
                              'Individual Income', 'Corporate Income', 'Motor Vehicle License', 'Other Taxes', 
                              'Current Charges', 'Interest Earnings', 'Special Assessments', 'Sale Of Property', 'Other General Revenue']},
        "Expenditures": {
            "Expenditures":[
                "Intergovernmental Expenditure", "Current Operations", "Capital Outlay",
                "Assistance And Subsidies", "Interest On Debt", "Insurance Benefits And Repayments"],
            "Expenditure by Function": [
                    "Intergovernmental Expenditure", "Education", "Libraries", "Public Welfare", "Hospitals", 
                    "Health", "Employment Security Administration", "Veterans' Services",
                    "Highways", "Air Transportation (Airports)", "Parking Facilities", "Sea And Inland Port Facilities",
                    "Police Protection", "Fire Protection", "Correction", "Protective Inspection And Regulation",
                    "Natural Resources", "Parks And Recreation", "Housing And Community Development", "Sewerage",
                    "Solid Waste Management", "Financial Administration","Judicial And Legal", "General Public Buildings",
                    "Other Governmental Administration", "Interest On General Debt", "Miscellaneous Commercial Activities",
                    "Other And Unallocable", "Utility Expenditure", "Liquor Store Expenditure", "Insurance Trust Expenditure"]},
    }

figs = {}
for revexp_key in areas.keys():
    for key, p_dfs in plot_dfs[revexp_key].items():
        figs[key] = {}
        for form, plot_df in p_dfs.items():# ["Level", "Real Level", "Percent of General Revenue", "Percent of GDP", "Real Value Per Capita"]:
            figs[key][form] = {}
            df = plot_df.copy()
            df.rename(columns = {k:k.replace("1", "").replace("2","").title() for k in df.keys()}, inplace = True)
            df = df.reset_index().melt(
                id_vars=["State","Year"],
                var_name="Name", 
                value_name="Value",
            )
            df = df.set_index(["State", "Year"])#.round(4)
            states = df.index.get_level_values("State").unique()

            for components_group, components in areas[revexp_key].items():


                figs[key][form][components_group] = {}

                for state in states:
                    plot_df = df.loc[state].reset_index()#.loc[start_year:].reset_index()
                    plot_df = plot_df[plot_df["Name"].isin(components)].dropna()
                    plot_df["Value"] = pd.to_numeric(plot_df["Value"])
                    title = f"<br>{key}<br>{form}: {state}<br>"
                    px_fig = px.area(
                        plot_df, x="Year", y="Value", color="Name")
                    px_fig.update_layout(title = dict(text = title,x = 0, xanchor = "left", yanchor = "bottom",y = 1),
                                         legend_title_text = components_group),
                    
                    figs[key][form][components_group][state] = px_fig
                figs[key][form][components_group]  = dict_of_figs_to_dropdown_fig(figs[key][form][components_group], 
                                                                                show_fig = False,
                                                                                use_sliders = True)
                figs[key][form][components_group].write_html(f"outputs/{key}/AreaPlots{revexp_key}{key}{form}{components_group}.html")
                

In [24]:
url = "https://www.fraserinstitute.org/sites/default/files/economic-freedom-of-north-america-2022-for-website-allgovs.xlsx"
storage_options = {'User-Agent': 'Mozilla/5.0'}
efna = pd.read_excel(url, storage_options=storage_options, sheet_name=None)
efnagdp = pd.read_csv("EFNAGDPTaxes.csv", parse_dates = ["Year"]).set_index(["State", "Year"]).sort_index()
employment_variables = ["Labor Force", "Unemployment Rate", "Labor Force Participation Rate"]


In [28]:
efna_unemployment = {"EFNA":efnagdp[[key for key in efnagdp if key not in employment_variables]],
                        "Employment":efnagdp[employment_variables]}
efna_unemployment["Employment"]["Unemployed"] = efna_unemployment["Employment"]["Labor Force"].mul(efna_unemployment["Employment"]["Unemployment Rate"]).div(100).astype(int)
efna_unemployment["Employment"]["Employed"] = efna_unemployment["Employment"]["Labor Force"].sub(efna_unemployment["Employment"]["Unemployed"]).astype(int)
efna_unemployment["EFNA"]

/tmp/ipykernel_30344/975105236.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_30344/975105236.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Unnamed: 0  Population           GDP  \
State Year                                               
AK    1985-01-01          39    532500.0           NaN   
      1986-01-01          40    544270.0           NaN   
      1987-01-01          41    539310.0           NaN   
      1988-01-01          42    541980.0           NaN   
      1989-01-01          43    547160.0           NaN   
...                      ...         ...           ...   
WY    2019-01-01        1945    580120.0  4.002950e+10   
      2020-01-01        1946    577660.0  3.670960e+10   
      2021-01-01        1947    579550.0  4.231910e+10   
      2022-01-01        1948    581630.0  4.978260e+10   
      2023-01-01        1949    584060.0  5.199050e+10   

                  Overall-scores(allgovs)  Overall-ranks(allgovs)  EFNA  \
State Year                                                                
AK    1985-01-01                     8.04                    44.0  8.04   
      1986-01-01                      NaN                     NaN   NaN   
      1987-01-01                      NaN                     NaN   NaN   
      1988-01-01                      NaN                     NaN   NaN   
      1989-01-01                      NaN                     NaN   NaN   
...                                   ...                     ...   ...   
WY    2019-01-01                     8.12                     8.0  8.12   
      2020-01-01                     7.97                    14.0  7.97   
      2021-01-01                      NaN                     NaN   NaN   
      2022-01-01                      NaN                     NaN   NaN   
      2023-01-01                      NaN                     NaN   NaN   

                  EFNA 1A  EFNA 1B  EFNA 1C  EFNA 1D  ...  T28  T29  T40  T41  \
State Year                                            ...                       
AK    1985-01-01     6.69     9.10     2.52     8.78  ...  NaN  NaN  NaN  NaN   
      1986-01-01      NaN      NaN      NaN      NaN  ...  NaN  NaN  NaN  NaN   
      1987-01-01      NaN      NaN      NaN      NaN  ...  NaN  NaN  NaN  NaN   
      1988-01-01      NaN      NaN      NaN      NaN  ...  NaN  NaN  NaN  NaN   
      1989-01-01      NaN      NaN      NaN      NaN  ...  NaN  NaN  NaN  NaN   
...                   ...      ...      ...      ...  ...  ...  ...  ...  ...   
WY    2019-01-01     8.39     9.75     1.73     9.53  ...  NaN  NaN  NaN  NaN   
      2020-01-01     8.34     9.74     1.47     9.23  ...  NaN  NaN  NaN  NaN   
      2021-01-01      NaN      NaN      NaN      NaN  ...  NaN  NaN  NaN  NaN   
      2022-01-01      NaN      NaN      NaN      NaN  ...  NaN  NaN  NaN  NaN   
      2023-01-01      NaN      NaN      NaN      NaN  ...  NaN  NaN  NaN  NaN   

                  T50  T51  T53  T99  TotalTax  TotalTax/GDP  
State Year                                                    
AK    1985-01-01  NaN  NaN  NaN  NaN       NaN           NaN  
      1986-01-01  NaN  NaN  NaN  NaN       NaN           NaN  
      1987-01-01  NaN  NaN  NaN  NaN       NaN           NaN  
      1988-01-01  NaN  NaN  NaN  NaN       NaN           NaN  
      1989-01-01  NaN  NaN  NaN  NaN       NaN           NaN  
...               ...  ...  ...  ...       ...           ...  
WY    2019-01-01  NaN  NaN  NaN  NaN       NaN           NaN  
      2020-01-01  NaN  NaN  NaN  NaN       NaN           NaN  
      2021-01-01  NaN  NaN  NaN  NaN       NaN           NaN  
      2022-01-01  NaN  NaN  NaN  NaN       NaN           NaN  
      2023-01-01  NaN  NaN  NaN  NaN       NaN           NaN  

[1950 rows x 60 columns]

In [6]:
income = pd.read_csv("PersonalIncomeStateData/CAINC5N__ALL_AREAS_2001_2022.csv", encoding="latin1", low_memory=False)
income["State"] = income["GeoName"].str.extract(r"([A-Z]{2})")
income["GeoFIPS"] = income["GeoFIPS"].str.replace('"', '')
income['StateFIPS'] = income["GeoFIPS"].str.extract(r"(\d{2})")
income["CountyFIPS"] = income["GeoFIPS"].str[-3:].astype(str)
income["County"] = income["GeoName"].str.split(",").str[0]
income.dropna(subset = ["State"], inplace = True)
income.rename(columns = {"Description": "Name"}, inplace = True)

for year in range(2001, 2023):
    year = str(year)
    income[year] = pd.to_numeric(income[year], errors = "coerce")
    income[year][income["Unit"] == income["Unit"].unique()[0]] = income[year][income["Unit"] == income["Unit"].unique()[0]].mul(10**3)

income["Unit"] = income["Unit"].str.replace("Thousands of dollars", "Dollars")
for orig_str, new_str in {"Thousands of dollars": "Dollars", "thousands of dollars": "Dollars", "dollars": "Dollars"}.items():
    income["Name"] = income["Name"].str.replace(orig_str, new_str)
income["Name"] = income["Name"].str.rstrip()
income_names = list(income["Name"].unique())
melt_df = income[["State", "County", "Name", "Unit"] + [str(y) for y in range(2001,2023)]].melt(id_vars=["State", "County", "Name", "Unit"], var_name="Year", value_name="Value")
melt_df = melt_df.set_index(["State", "County", "Name", "Year"])[["Value"]]
melt_df["Value"] = pd.to_numeric(melt_df["Value"], errors = "coerce")
state_income = melt_df.groupby(["State", "Year", "Name"]).sum()
total_wages_salaries = state_income[state_income.index.get_level_values("Name") == " Wages and salaries"].add(state_income[state_income.index.get_level_values("Name") == " Supplements to wages and salaries"].values)
total_wages_salaries.reset_index(inplace=True)
total_wages_salaries['Name'] = "Total Wages and Salaries"
income_names = income_names + ["Total Wages and Salaries"]
total_wages_salaries.set_index(["State", "Year", "Name"], inplace = True)
state_income = pd.concat([state_income, total_wages_salaries])
state_income = state_income.reset_index()
state_income["Year"] = state_income["Year"].astype(str)
state_income.set_index(["State", "Year", "Name"], inplace = True)
# for name in ["Population", "EFNA", "Unemployment Rate", "Labor Force Participation Rate"]:
#     tdf = pd.DataFrame(efnagdp[name], columns = [name]).rename(columns = {name: "Value"})

#     tdf["Name"] = name
#     tdf = tdf.reset_index()
#     tdf["Year"] = tdf["Year"].dt.year.astype(str)
#     tdf.set_index(["State", "Year", "Name"], inplace = True)
#     state_income = pd.concat([state_income, tdf])

pi_dfs = {k: state_income.copy() for k in ["Level", "Real Level", "Percent of Personal Income", "Real Value Per Capita"]}
cpi_str_index = cpi.reset_index().copy()
cpi_str_index["Year"] = pd.to_datetime(cpi_str_index["Year"]).dt.year.astype(str)
cpi_str_index.set_index("Year", inplace = True)
pi_dfs["Level"]["Value"] = omit_variables(pi_dfs["Level"]["Value"], ["Per capita personal income (Dollars)"])
pi_dfs["Real Level"]["Value"] = pi_dfs["Level"]["Value"].div(cpi_str_index["CPI"], level = "Year", axis = 0)
pi_dfs["Real Level"]["Value"] = omit_variables(pi_dfs["Real Level"]["Value"], ["Population (persons) 2/"])
pi_dfs["Percent of Personal Income"]["Value"] = state_income["Value"].div(state_income["Value"].xs("Personal income (Dollars)", level = "Name")).mul(100)
pi_dfs["Percent of Personal Income"]["Value"] = omit_variables(pi_dfs["Percent of Personal Income"]["Value"], 
                                                               ["Population (persons) 2/"])
pi_dfs["Real Value Per Capita"]["Value"] = pi_dfs["Real Level"]["Value"].div(state_income["Value"].xs("Population (persons) 2/", level = "Name"))
pi_dfs["Real Value Per Capita"]["Value"] = omit_variables(pi_dfs["Real Value Per Capita"]["Value"], 
                                                          ["Per capita personal income (Dollars)", "Population (persons) 2/"])

/tmp/ipykernel_30344/3705999658.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_30344/3705999658.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_30344/3705999658.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_30344/3705999658.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas

In [7]:
efnagdp.keys()

Index(['Unnamed: 0', 'Population', 'GDP', 'Labor Force', 'Unemployment Rate',
       'Overall-scores(allgovs)', 'Overall-ranks(allgovs)', 'EFNA', 'EFNA 1A',
       'EFNA 1B', 'EFNA 1C', 'EFNA 1D', 'EFNA 2A', 'EFNA 2B', 'EFNA 2Bi',
       'EFNA 2Bii', 'EFNA 2C', 'EFNA 2D', 'EFNA 3A', 'EFNA 3Ai', 'EFNA 3Aii',
       'EFNA 3Aiii', 'EFNA 3Aiv', 'EFNA 3B', 'EFNA 3C', 'EFNA 4', 'EFNA 5',
       'EFNA 6', 'Labor Force Participation Rate', 'GDP Per Capita', 'Log GDP',
       'GDP % Growth', 'Log Population', 'Population % Growth',
       'Log GDP Per Capita', 'GDP Per Capita % Growth', 'T01', 'T09', 'T10',
       'T11', 'T12', 'T13', 'T14', 'T15', 'T16', 'T19', 'T20', 'T21', 'T22',
       'T23', 'T24', 'T25', 'T27', 'T28', 'T29', 'T40', 'T41', 'T50', 'T51',
       'T53', 'T99', 'TotalTax', 'TotalTax/GDP'],
      dtype='object')

In [8]:
try:
    os.mkdir("outputs/PersonalIncome")
except:
    pass
names = list(pi_dfs["Level"].index.get_level_values("Name").unique())
names = [n for n in names if "    " not in n]
areas = {str(i):{} for i in range(3, -1, -1)}
for num_spaces in areas.keys():
    areas[num_spaces] = [n for n in names if " " * int(num_spaces) in n[:int(num_spaces)]]
    names= [n for n in names if n not in areas[num_spaces] ]
    
figs = {}
key = "Personal Income"

for form, plot_df in pi_dfs.items():# ["Level", "Real Level", "Percent of General Revenue", "Percent of GDP", "Real Value Per Capita"]:
    figs[form] = {}
    df = plot_df.copy()

    states = df.index.get_level_values("State").unique()

    for components_group, components in areas.items():
        figs[form][components_group] = {}
        for state in states:
            plot_df = df.loc[state].reset_index()#.loc[start_year:].reset_index()
            plot_df = plot_df[plot_df["Name"].isin(components)].dropna()
            plot_df["Value"] = pd.to_numeric(plot_df["Value"])
            title_key = f"{components_group} {key}<br>{form}<br>{state}"
            px_fig = px.area(
                plot_df, x="Year", y="Value", color="Name", title = title_key)

            figs[form][components_group][state] = px_fig
        figs[form][components_group]  = dict_of_figs_to_dropdown_fig(figs[form][components_group], 
                                                                        show_fig = False,
                                                                        use_sliders = True)
        figs[form][components_group].write_html(f"outputs/PersonalIncome/AreaPlots{key}{form}{components_group}.html".replace(" ", ""))

In [9]:
# pi_keys = {k:["Unemployment Rate", "EFNA", "Population"] + [n for n in income_names if n in v.dropna().index.get_level_values("Name").unique()] for k, v in pi_dfs.items()}
pi_keys = {k: [n for n in income_names if n in v.dropna().index.get_level_values("Name").unique()] for k, v in pi_dfs.items()}
pi_dfs = {k:v.reset_index().pivot_table(index=['State', 'Year'], columns='Name', values='Value')[pi_keys[k]] for k,v in pi_dfs.items()}


scatter_figs = {}
for key, df in pi_dfs.items():
    title = f"Personal Income<br>{key}"
    map_figs = {}

    html_path = f"outputs/PersonalIncome/ScatterPlotsPersonalIncome{key}.html"
    create_scatter_dropdown(df, regions_df=regions_df, title = title,
                            filename = html_path,  
                            show_fig = False)
    for name in df.keys():
        map_figs[name] = create_map(df.reset_index(), name, time_name = "Year", title=title)
    combined_map_fig = combine_map_figs(map_figs, title = title)
    html_path = f"outputs/PersonalIncome/MapPlotsPersonalIncome{key}.html"
    combined_map_fig.write_html(html_path)

regions_df = pd.read_csv("USCensusRegions.csv")#.set_index("State")
filename = f"outputs/PersonalIncome/LinePlotsStatePI.html"
fig = aggregated_line_dropdown(pi_dfs, regions_df)
fig.write_html(filename, config=dict(displayModeBar=True))

In [29]:
efna_description = efna["Index Components"]["The Areas and Components of the Economic Freedom of North America Index"].dropna()
codes = efna_description.str[:3]
for char in [":", "."]: 
    codes = codes.str.replace(char, "")
maxef = 6
codes = "EFNA " + codes
efna["Index Components"]["Code"] = codes.str.strip()
efna_description = efna["Index Components"].copy().dropna()
efna_description = efna_description[~(efna_description["Code"].str.contains("i")| efna_description["Code"].str.contains("v"))]
efna_description.rename(columns = {"The Areas and Components of the Economic Freedom of North America Index": "Description"}, inplace = True)
efna_description.set_index("Code", inplace = True)
efna_description.loc["EFNA 2B"] = '2B. Top Marginal Income Tax Rate Attributes'
keys = list(efnagdp.keys())
end_index = keys.index(f"EFNA {maxef}")

efna_unemployment["EFNA"] = efnagdp[keys[3:end_index+1]]
efna_unemployment["EFNA"]  = efna_unemployment["EFNA"].dropna(how = "all", axis = 0)
efna_sub_keys = {f"EFNA {i}":[k for k in efna_unemployment["EFNA"].keys() if f"EFNA {i}" in k and "i" not in k] for i in range(1,maxef+1)}
for i in range(1,7):
    efna_unemployment["EFNA"] [f"EFNA {i}"] = efna_unemployment["EFNA"] [efna_sub_keys[f"EFNA {i}"]].mean(axis = 1).round(3)
efna_sub_keys["Major EFNA Components"] = [f"EFNA {i}" for i in range(maxef+1)]

In [36]:
for dct_key, df in efna_unemployment.items():
    try:
        os.mkdir(f"outputs/{dct_key}")
    except:
        pass

    figs = {}
    df = df.reset_index().melt(id_vars=["State","Year"],
        var_name="Name", 
        value_name="Value",
        ).set_index(["State", "Year"])
    if dct_key == "EFNA":
        for key, keys in efna_sub_keys.items():
            if len(keys) > 1:
                components_group = key
                components = keys
                figs[components_group] = {}
                # df = efna_df.copy()
                # df = df.reset_index().melt(id_vars=["State","Year"],
                #     var_name="Name", 
                #     value_name="Value",
                #     )
                # df = df.set_index(["State", "Year"])#.round(4)
                states = df.index.get_level_values("State").unique()

                for state in states:
                    plot_df = df.loc[state].reset_index()#.loc[start_year:].reset_index()
                    plot_df = plot_df[plot_df["Name"].isin(components)].dropna().sort_values("Name")
                    plot_df["Name"] = plot_df["Name"].apply(lambda x: efna_description.loc[x]["Description"])
                    plot_df["Value"] = pd.to_numeric(plot_df["Value"]).div(len(components))
                    title_key = f"{components_group} <br>{state}"
                    px_fig = px.area(
                        plot_df, x="Year", y="Value", color="Name", title = title_key)

                    figs[components_group][state] = px_fig
                figs[components_group]  = dict_of_figs_to_dropdown_fig(figs[components_group], 
                                                                                show_fig = False,
                                                                                use_sliders = True)
                figs[components_group].write_html(f"outputs/{dct_key}/AreaPlots{components_group}.html")
    else:
        pass
    plot_df = pd.pivot_table(df.reset_index(), 
                             index = ["State", "Year"], 
                             columns = "Name", 
                             values = "Value")
    map_figs = {}
    for name in plot_df.keys():
        map_figs[name] = create_map(plot_df.reset_index(), name, time_name = "Year", title=name)

    combined_map_fig = combine_map_figs(map_figs, title = key)
    html_path = f"outputs/{dct_key}/MapPlots{dct_key}.html"
    combined_map_fig.write_html(html_path)

/home/jamescaton/anaconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

/home/jamescaton/anaconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

/home/jamescaton/anaconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the 

KeyError: 'Value'

In [37]:
df

Labor Force  Unemployment Rate  \
State Year                                         
AK    1985-01-01    249600.75               9.84   
      1986-01-01    254813.67              11.10   
      1987-01-01    249574.50              10.14   
      1988-01-01    250991.25               8.73   
      1989-01-01    257917.00               7.10   
...                       ...                ...   
WY    2019-01-01    293997.33               3.71   
      2020-01-01    293605.75               5.86   
      2021-01-01    288769.00               4.54   
      2022-01-01    292663.92               3.42   
      2023-01-01    295206.42               2.92   

                  Labor Force Participation Rate  Unemployed  Employed  
State Year                                                              
AK    1985-01-01                       46.873380       24560    225040  
      1986-01-01                       46.817512       28284    226529  
      1987-01-01                       46.276631       25306    224268  
      1988-01-01                       46.310058       21911    229080  
      1989-01-01                       47.137400       18312    239605  
...                                          ...         ...       ...  
WY    2019-01-01                       50.678710       10907    283090  
      2020-01-01                       50.826741       17205    276400  
      2021-01-01                       49.826417       13110    275659  
      2022-01-01                       50.317886       10009    282654  
      2023-01-01                       50.543852        8620    286586  

[1950 rows x 5 columns]

In [12]:
# regions_df = pd.read_csv("USCensusRegions.csv")#.set_index("State")
rename_df = efna["Index Components"]["The Areas and Components of the Economic Freedom of North America Index"].dropna()
rename_df.index = "EFNA " + rename_df.str.split(" ").str[0].str.replace(":", "").str.replace(".","")
rename_df.loc["EFNA 2B"] = '2B. Top Marginal Income Tax Rate Attributes'
plot_df = efna_df.rename(columns = {k:rename_df.loc[k] for k in rename_df.iloc[1:].index})
fig = line_dropdown(plot_df.sort_index(axis =1), regions_df)
fig.write_html("outputs/EFNA/EFNALinePlot.html")


/home/jamescaton/anaconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [13]:
retire = pd.read_csv("NDRetirementSystemData.csv", parse_dates = ["Fiscal Year"]).rename(columns = {"Fiscal Year": "Year"}).set_index("Year")
retire['Cash Flow (millions)'].div(retire["Net Assets (billions)"]).div(1000)
retire["Net Assets Growth Rate"] = retire["Net Assets (billions)"].pct_change().mul(100)
retire["Net Assets Less Cash Flow Growth Rate"] = retire["Net Assets Growth Rate"].sub(retire["ND Cash Flow as Percent Assets"])


In [14]:
from homebrewedFunctions.functions import *
# https://publicplansdata.org/public-plans-database/
regions_df = pd.read_csv("USCensusRegions.csv")#.set_index("State")

us_retire = pd.read_csv(
    "ppd-data-latest.csv", 
    encoding = "latin1", low_memory = False, parse_dates = ["fy"]).rename(
    columns = {"fy": "Year"}).set_index(["StateName","PlanFullName", "Year"])
states = pd.read_csv("stateFIPSAbbrev.csv", index_col = "State").rename(columns={"Unnamed: 0": "StateAbbrev"})
us_retire["State"] = us_retire.index.get_level_values("StateName").map(lambda x: states.loc[x]["StateAbbrev"], 
                                                                       na_action = "ignore")
for act_al in ["ActAssets_GASB", "ActLiabilities_GASB"]:
    us_retire[act_al] = us_retire[act_al].mul(10**3)
# us_retire = us_retire.dropna(subset = ["State"]).groupby(["StateAbbrev", "Year"])[["ActAssets_GASB", "ActLiabilities_GASB"]].sum()
us_retire["PercentFundedGASB"] = us_retire["ActAssets_GASB"].div(us_retire["ActLiabilities_GASB"]).mul(100)
us_retire["UnfundedAmount"] = us_retire["ActLiabilities_GASB"].sub(us_retire["ActAssets_GASB"])
us_retire["UnfundedAmountPerMember"] = us_retire["UnfundedAmount"].div(us_retire["TotMembership"])
us_retire = us_retire.reset_index().set_index(["State", "PlanFullName", "Year"])[["ActAssets_GASB", "ActLiabilities_GASB", "PercentFundedGASB", 
                                    "TotMembership", "UnfundedAmount", "UnfundedAmountPerMember"]]
us_retire = us_retire[~us_retire.index.get_level_values("State").isin([np.nan])]
us_retire.sort_index(inplace = True)


plot_dfs = {}
for state in us_retire.index.get_level_values("State").unique():
    plot_df = us_retire.loc[state].sort_index().reset_index()
    plot_df["Year"] = plot_df["Year"].dt.year
    plot_df.set_index(["PlanFullName", "Year"], inplace = True)


    plot_dfs[state] = plot_df.copy()


In [15]:
folder = "StateRetirementPlans"
path = f"outputs/{folder}"
try:
    os.mkdir(path)
except:
    pass
figs = {}
for i, (state, plot_df) in enumerate(plot_dfs.items()):
    # total_plot_df = p_df.groupby("Year").sum()
    # total_plot_df["PercentFundedGASB"] = total_plot_df["ActAssets_GASB"].div(total_plot_df["ActLiabilities_GASB"]).mul(100)   
    # plot_df = pd.concat([p_df.reset_index(), total_plot_df.reset_index()], axis = 0).set_index(["PlanFullName", "Year"])

    keys = list(plot_df.keys())
    first_key = keys[0]
    # if i >5:
    #     continue
    fig = px.line(plot_df.reset_index(), x = "Year", y = first_key, color = "PlanFullName")
    fig.update_layout(legend = dict(x = 0, xanchor = "left", 
                                    y = 1, yanchor = "bottom",
                                    orientation = "h")
    )

    y_buttons = []
    for col in keys:
        if is_numeric_dtype(plot_df[col]):
            y_buttons.append(
                dict(
                    args=[
                        {"y": [plot_df.loc[plan][col].dropna(axis = 0) for plan in plot_df.index.get_level_values('PlanFullName').unique()]},
                        {"yaxis.title.text": col}
                    ],
                    label=col,
                    method="update"
                )
            )
    menu_font =dict(size=20)

    fig.update_layout(
        updatemenus=[
            dict(
                buttons=y_buttons,
                direction="down",
                showactive=True,
                x=1,
                xanchor="right",
                y=1.25,
                yanchor="bottom",
            ),
            dict(
                type="buttons",
                direction="left",
                buttons=[
                    dict(
                        args=[{"yaxis.type": "linear",
                            "font":menu_font}],
                        label="Linear Y",
                        method="relayout"
                    ),
                    dict(
                        args=[{"yaxis.type": "log",
                            "font":menu_font}],
                        label="Log Y",
                        method="relayout"
                    )
                ],
                x=0,
                xanchor="left",
                y=1.25,
                yanchor="bottom"
            )
        ])
    fig.update_layout(
        # updatemenus=create_menus(first_key),
        margin=dict(t=200),
        font=dict(size=20),
        clickmode='event+select',
        hovermode='closest',
        title = dict(text=f"{state}", x=1, y = 1, xanchor='right', yanchor = 'bottom'),         
        legend_title_text = "Plan Name")
    figs[state] = fig
agg_fig = dict_of_figs_to_dropdown_fig(figs, show_fig=False, use_sliders=True)
filename = f"{path}/LinePlots{folder}.html"
agg_fig.write_html(filename, include_plotlyjs='cdn')

# with open(filename, "r") as f:
#     text = f.read()
# html_script = """
#     <script>
#         // After the plot is rendered
#         document.addEventListener('DOMContentLoaded', function() {
#             // Get the plotly figure
#             var plotElement = document.getElementsByClassName('plotly-graph-div')[0];

#             // Update the legend to enable scrolling
#             Plotly.relayout(plotElement, {
#                 'legend': {
#                     'scroll': true,
#                     'scrollheight': 150  // Set your desired max height in pixels
#                 }
#             });
#         });
#     </script>
# """
# text = text.replace("</body>", html_script + "</body>")
# with open(filename, "w") as f:
#     f.write(text)

Unnamed: 0  Population           GDP  Labor Force  \
State Year                                                            
AK    1985-01-01          39    532500.0           NaN    249600.75   
      1986-01-01          40    544270.0           NaN    254813.67   
      1987-01-01          41    539310.0           NaN    249574.50   
      1988-01-01          42    541980.0           NaN    250991.25   
      1989-01-01          43    547160.0           NaN    257917.00   
...                      ...         ...           ...          ...   
WY    2019-01-01        1945    580120.0  4.002950e+10    293997.33   
      2020-01-01        1946    577660.0  3.670960e+10    293605.75   
      2021-01-01        1947    579550.0  4.231910e+10    288769.00   
      2022-01-01        1948    581630.0  4.978260e+10    292663.92   
      2023-01-01        1949    584060.0  5.199050e+10    295206.42   

                  Unemployment Rate  Overall-scores(allgovs)  \
State Year                                                     
AK    1985-01-01               9.84                     8.04   
      1986-01-01              11.10                      NaN   
      1987-01-01              10.14                      NaN   
      1988-01-01               8.73                      NaN   
      1989-01-01               7.10                      NaN   
...                             ...                      ...   
WY    2019-01-01               3.71                     8.12   
      2020-01-01               5.86                     7.97   
      2021-01-01               4.54                      NaN   
      2022-01-01               3.42                      NaN   
      2023-01-01               2.92                      NaN   

                  Overall-ranks(allgovs)  EFNA  EFNA 1A  EFNA 1B  ...  T40  \
State Year                                                        ...        
AK    1985-01-01                    44.0  8.04     6.69     9.10  ...  NaN   
      1986-01-01                     NaN   NaN      NaN      NaN  ...  NaN   
      1987-01-01                     NaN   NaN      NaN      NaN  ...  NaN   
      1988-01-01                     NaN   NaN      NaN      NaN  ...  NaN   
      1989-01-01                     NaN   NaN      NaN      NaN  ...  NaN   
...                                  ...   ...      ...      ...  ...  ...   
WY    2019-01-01                     8.0  8.12     8.39     9.75  ...  NaN   
      2020-01-01                    14.0  7.97     8.34     9.74  ...  NaN   
      2021-01-01                     NaN   NaN      NaN      NaN  ...  NaN   
      2022-01-01                     NaN   NaN      NaN      NaN  ...  NaN   
      2023-01-01                     NaN   NaN      NaN      NaN  ...  NaN   

                  T41  T50  T51  T53  T99  TotalTax  TotalTax/GDP  Unemployed  \
State Year                                                                      
AK    1985-01-01  NaN  NaN  NaN  NaN  NaN       NaN           NaN       24560   
      1986-01-01  NaN  NaN  NaN  NaN  NaN       NaN           NaN       28284   
      1987-01-01  NaN  NaN  NaN  NaN  NaN       NaN           NaN       25306   
      1988-01-01  NaN  NaN  NaN  NaN  NaN       NaN           NaN       21911   
      1989-01-01  NaN  NaN  NaN  NaN  NaN       NaN           NaN       18312   
...               ...  ...  ...  ...  ...       ...           ...         ...   
WY    2019-01-01  NaN  NaN  NaN  NaN  NaN       NaN           NaN       10907   
      2020-01-01  NaN  NaN  NaN  NaN  NaN       NaN           NaN       17205   
      2021-01-01  NaN  NaN  NaN  NaN  NaN       NaN           NaN       13110   
      2022-01-01  NaN  NaN  NaN  NaN  NaN       NaN           NaN       10009   
      2023-01-01  NaN  NaN  NaN  NaN  NaN       NaN           NaN        8620   

                  Employed  
State Year                  
AK    1985-01-01    225040  
      1986-01-01    226529  
      1987-01-01    224268  
      1988-01-01    229080  
      1989-01-01  